In [355]:
%matplotlib inline
import numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns
import pickle, os, gc, re



In [356]:
BES_data_folder = '../BES_analysis_data/'

In [357]:
Pollbase_folder = create_subdir(BES_data_folder,"Pollbase")
xl = pd.ExcelFile(Pollbase_folder+'PollBase-Q4-2019.xlsx')

In [8]:
xl.sheet_names

['Introduction',
 '43-45',
 '45-50',
 '50-51',
 '51-55',
 '55-59',
 '59-64',
 '64-66',
 '66-70',
 '70-74',
 '74',
 '74-79',
 '79-83',
 '83-87',
 '87-92',
 '92-97',
 '97-01',
 '01-05',
 '05-10',
 '10-15',
 '15-17',
 '17-19',
 '19-',
 'Monthly average',
 'By election polls',
 '2014 Euros',
 'Gallup ratings 1989-2010',
 'YouGov rolling ratings 2006-7']

In [310]:
df_list = []

In [311]:
sheet_name = xl.sheet_names[1]
print(sheet_name)
df = xl.parse(xl.sheet_names[1])

df = df[['Year', 'Month', 'Fieldwork', 'Date', 'Polling', 'Publisher', 'Con',
       'Unnamed: 7', 'Lab', 'Unnamed: 9', 'LD', 'Unnamed: 11', 'Con lead',
       'Unnamed: 13']]
df.columns = ['Year', 'Month', 'Fieldwork', 'Publication Day', 'Polling', 'Publisher', 'Con',
       'Con Error', 'Lab', 'Lab Error', 'LD', 'LD Error', 'Con lead',
       'Con lead Error']

# drop empty rows
df.drop(df.index[df.isnull().all(axis=1)],inplace=True)

df["Year"] = df["Year"].fillna( method ='ffill',axis=0 ).astype('int')
df["Month"] = df["Month"].fillna( method ='ffill',axis=0 )

df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Start_Day" ,df["Fieldwork"].apply(lambda x: x.split("-")[0] if pd.notnull(x) else np.nan).astype("float").astype("Int64") )
df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Finish_Day" ,df["Fieldwork"].apply(lambda x: x.split("-")[1] if pd.notnull(x) else np.nan).astype("float").astype("Int64") )          

df.drop("Fieldwork",axis=1,inplace=True)




# 8	1945	Jun?	NaN	14/6 and 15/7
df.loc[8,"Month"] = "Jun"
df.loc[8,"Publication Day"] = 14 # 14/15???

# 1945GE = 5 July 1945
df.loc[df["Polling"]=="Result",["Year","Month","Fieldwork_Start_Day","Fieldwork_Finish_Day","Publication Day"]]=\
    [1945,"July",5,5,5]

df["Publication Day"] = df["Publication Day"].astype("float").astype("Int64")
df_list.append(df)
df

43-45


,Year,Month,Fieldwork_Start_Day,Fieldwork_Finish_Day,Publication Day,Polling,Publisher,Con,Con Error,Lab,Lab Error,LD,LD Error,Con lead,Con lead Error
0,1943,Jun,NaN,NaN,NaN,Gallup,News Chron,36.0,NaN,44.0,NaN,10.5,NaN,-8.0,NaN
1,1943,Jul,NaN,NaN,NaN,Gallup,News Chron,33.0,-3.0,47.5,3.5,11.0,0.5,-14.5,-6.5
2,1943,Dec,NaN,NaN,NaN,Gallup,News Chron,31.5,-1.5,46.5,-1.0,11.5,0.5,-15.0,-0.5
3,1944,Jan,NaN,NaN,NaN,Gallup,News Chron,28.0,-3.5,45.0,-1.5,12.0,0.5,-17.0,-2.0
4,1945,Feb,NaN,NaN,NaN,Gallup,News Chron,27.5,-0.5,47.5,2.5,12.5,0.5,-20.0,-3.0
5,1945,Apr,NaN,NaN,NaN,Gallup,News Chron,28.0,0.5,47.0,-0.5,14.0,1.5,-19.0,1.0
6,1945,May,NaN,NaN,NaN,Gallup,News Chron,33.0,5.0,45.0,-2.0,15.0,1.0,-12.0,7.0
7,1945,Jun,NaN,NaN,NaN,Gallup,News Chron,37.0,4.0,45.0,0.0,15.0,0.0,-8.0,4.0
8,1945,Jun,NaN,NaN,14,Centre of Public Opinion,Daily Express,45.0,NaN,45.0,NaN,12.0,NaN,0.0,NaN
9,1945,Jun,24,27,4,Gallup,News Chron,41.0,4.0,47.0,2.0,10.5,-5.0,-6.0,2.0


In [314]:
sheet_name = xl.sheet_names[2]
print(sheet_name)
df = xl.parse(sheet_name)

df = df[['Year', 'Month', 'Fieldwork', 'Date', 'Polling', 'Publisher', 'Con',
       'Unnamed: 7', 'Lab', 'Unnamed: 9', 'LD', 'Unnamed: 11', 'Con lead',
       'Unnamed: 13',]]
df.columns = ['Year', 'Month', 'Fieldwork', 'Publication Day', 'Polling', 'Publisher', 'Con',
       'Con Error', 'Lab', 'Lab Error', 'LD', 'LD Error', 'Con lead',
       'Con lead Error']
        

# drop empty rows
df.drop(df.index[df.isnull().all(axis=1)],inplace=True)

df["Year"] = df["Year"].fillna( method ='ffill',axis=0 ).astype('int')
df["Month"] = df["Month"].fillna( method ='ffill',axis=0 )

df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Start_Day" ,df["Fieldwork"].apply(lambda x: x.split("-")[0] if pd.notnull(x) else np.nan).astype("float").astype("Int64") )
df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Finish_Day" ,df["Fieldwork"].apply(lambda x: x.split("-")[1] if pd.notnull(x) else np.nan).astype("float").astype("Int64") )          
df.drop("Fieldwork",axis=1,inplace=True)



## Manual fixes("")

# 1950GE = 23 February 1950
df.loc[df["Polling"]=="Result",["Year","Month","Fieldwork_Start_Day","Fieldwork_Finish_Day","Publication Day"]]=\
    [1950,"Feb",23,23,23]

df["Publication Day"] = df["Publication Day"].astype("Int64")
df_list.append(df)
df        

45-50


,Year,Month,Fieldwork_Start_Day,Fieldwork_Finish_Day,Publication Day,Polling,Publisher,Con,Con Error,Lab,Lab Error,LD,LD Error,Con lead,Con lead Error
0,1946,Jan,NaN,NaN,NaN,Gallup,News Chron,32.0,NaN,52.5,NaN,11.0,NaN,-20.5,NaN
1,1946,May,NaN,NaN,NaN,Gallup,News Chron,40.0,8.0,43.5,-9.0,13.0,2.0,-3.5,17.0
2,1947,Jan,NaN,NaN,NaN,Gallup,News Chron,41.0,1.0,44.5,1.0,12.0,-1.0,-3.5,0.0
3,1947,Mar,NaN,NaN,NaN,Gallup,News Chron,43.5,2.5,43.5,-1.0,10.5,-1.5,0.0,3.5
4,1947,Jun,NaN,NaN,NaN,Gallup,News Chron,42.5,-1.0,42.5,-1.0,12.5,2.0,0.0,0.0
5,1947,Jul,NaN,NaN,NaN,Gallup,News Chron,42.5,0.0,42.5,0.0,12.5,0.0,0.0,0.0
6,1947,Aug,NaN,NaN,NaN,Gallup,News Chron,44.5,2.0,41.0,-1.5,11.0,-1.5,3.5,3.5
7,1947,Sep,NaN,NaN,NaN,Gallup,News Chron,44.5,0.0,39.5,-1.5,11.5,0.5,5.0,1.5
8,1947,Nov,NaN,NaN,NaN,Gallup,News Chron,50.5,6.0,38.0,-1.5,9.0,-2.5,12.5,7.5
9,1948,Jan,NaN,NaN,NaN,Gallup,News Chron,44.5,-6.0,43.5,5.5,10.5,1.5,1.0,-11.5


In [124]:
def get_fieldwork_days(fw_str, part):
    if pd.isnull(fw_str):
        return np.nan
    elif isinstance(fw_str,int):
        return fw_str
    elif re.match("(\d+|\?)-(\d+|\?)",fw_str):
        temp = [x if x != "?" else np.nan for x in fw_str.split("-") ]
        return temp[part]
    elif re.match("\d+",fw_str):
        return fw_str
    


In [315]:
sheet_name = xl.sheet_names[3]
print(sheet_name)
df = xl.parse(sheet_name)
df
df = df[['Year', 'Month', 'Fieldwork', 'Date', 'Polling', 'Publisher', 'Con',
       'Unnamed: 7', 'Lab', 'Unnamed: 9', 'LD', 'Unnamed: 11', 'Con lead',
       'Unnamed: 13', 'Con good', 'Con bad',
       'Con net', 'Lab good', 'Lab bad', 'Lab net', 'LD good', 'LD bad',
       'LD net', 'Question', 'Govt good', 'Govt bad',
       'Govt net', 'Question.1']]
# 'Con good', 'Con bad',
#        'Con net', 'Lab good', 'Lab bad', 'Lab net', 'LD good', 'LD bad',
#        'LD net', 'Question', 'Govt good', 'Govt bad',
#        'Govt net', 'Question.1'
df.columns = ['Year', 'Month', 'Fieldwork', 'Publication Day', 'Polling', 'Publisher', 'Con',
       'Con Error', 'Lab', 'Lab Error', 'LD', 'LD Error', 'Con lead',
       'Con lead Error',
       'Leader: Con good', 'Leader: Con bad',
       'Leader: Con net', 'Leader: Lab good', 'Leader: Lab bad', 'Leader: Lab net',
       'Leader: LD good', 'Leader: LD bad', 'Leader: LD net', 'Leader: Question', 'Govt good', 'Govt bad',
       'Govt net', 'Govt Question']
        

# drop empty rows
df.drop(df.index[df.isnull().all(axis=1)],inplace=True)

df["Year"] = df["Year"].fillna( method ='ffill',axis=0 ).astype('int')
df["Month"] = df["Month"].fillna( method ='ffill',axis=0 )

df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Start_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 0) ).astype("float").astype("Int64") )
df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Finish_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 1) ).astype("float").astype("Int64") )          
df.drop("Fieldwork",axis=1,inplace=True)


## Manual fixes("")

# 1951GE = 25 October 1951
df.loc[df["Polling"]=="Result",["Year","Month","Fieldwork_Start_Day","Fieldwork_Finish_Day","Publication Day"]]=\
    [1951,"Oct",25,25,25]

df["Publication Day"] = df["Publication Day"].astype("Int64")
df_list.append(df)
df        

50-51


,Year,Month,Fieldwork_Start_Day,Fieldwork_Finish_Day,Publication Day,Polling,Publisher,Con,Con Error,Lab,...,Leader: Lab bad,Leader: Lab net,Leader: LD good,Leader: LD bad,Leader: LD net,Leader: Question,Govt good,Govt bad,Govt net,Govt Question
0,1950,Mar,NaN,NaN,NaN,Gallup,News Chron,43.5,NaN,45.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1950,Apr,NaN,NaN,NaN,Gallup,News Chron,45.5,2.0,47.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1950,May,NaN,NaN,NaN,Gallup,News Chron,43.5,-2.0,46.5,...,39.0,11.0,NaN,NaN,NaN,Do you approve or disapprove of X as Y?,39.0,51.0,-12.0,Are you satisfied or dissatisfied with the Gov...
3,1950,Jun,NaN,NaN,NaN,Gallup,News Chron,43.5,0.0,46.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1950,Jul,NaN,NaN,NaN,Gallup,News Chron,42.0,-1.5,43.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1950,Aug,NaN,NaN,NaN,Gallup,News Chron,44.5,2.5,46.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1950,Sep,NaN,NaN,NaN,Gallup,News Chron,43.0,-1.5,45.5,...,43.0,6.0,NaN,NaN,NaN,Do you approve or disapprove of X as Y?,44.0,46.0,-2.0,Are you satisfied or dissatisfied with the Gov...
7,1950,Oct,NaN,NaN,NaN,Gallup,News Chron,42.5,-0.5,45.0,...,40.0,7.0,NaN,NaN,NaN,Do you approve or disapprove of X as Y?,45.0,45.0,0.0,Are you satisfied or dissatisfied with the Gov...
8,1950,Nov,NaN,NaN,NaN,Gallup,News Chron,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1950,Dec,NaN,NaN,NaN,Gallup,News Chron,43.0,NaN,44.0,...,38.0,11.0,NaN,NaN,NaN,Do you approve or disapprove of X as Y?,38.0,51.0,-13.0,Are you satisfied or dissatisfied with the Gov...


In [316]:
sheet_name = xl.sheet_names[4]
print(sheet_name)
df = xl.parse(sheet_name)
# df
df = df[['Year', 'Month', 'Fieldwork', 'Date', 'Polling', 'Publisher', 'Con',
       'Unnamed: 7', 'Lab', 'Unnamed: 9', 'LD', 'Unnamed: 11', 'Con lead',
       'Unnamed: 13']]
# 'Con good', 'Con bad',
#        'Con net', 'Lab good', 'Lab bad', 'Lab net', 'LD good', 'LD bad',
#        'LD net', 'Question', 'Govt good', 'Govt bad',
#        'Govt net', 'Question.1'
df.columns = ['Year', 'Month', 'Fieldwork', 'Publication Day', 'Polling', 'Publisher', 'Con',
       'Con Error', 'Lab', 'Lab Error', 'LD', 'LD Error', 'Con lead',
       'Con lead Error']
        

# drop empty rows
df.drop(df.index[df.isnull().all(axis=1)],inplace=True)

df["Year"] = df["Year"].fillna( method ='ffill',axis=0 ).astype('int')
df["Month"] = df["Month"].fillna( method ='ffill',axis=0 )

df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Start_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 0) ).astype("float").astype("Int64") )
df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Finish_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 1) ).astype("float").astype("Int64") )          
df.drop("Fieldwork",axis=1,inplace=True)


## Manual fixes("")

# 1955GE = 26 May 1955
df.loc[df["Polling"]=="Result",["Year","Month","Fieldwork_Start_Day","Fieldwork_Finish_Day","Publication Day"]]=\
    [1955,"May",26,26,26]

df["Publication Day"] = df["Publication Day"].astype("Int64")
df_list.append(df)
df        

51-55


,Year,Month,Fieldwork_Start_Day,Fieldwork_Finish_Day,Publication Day,Polling,Publisher,Con,Con Error,Lab,Lab Error,LD,LD Error,Con lead,Con lead Error
0,1951,Nov,NaN,NaN,NaN,Gallup,News Chron,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1951,Dec,NaN,NaN,NaN,Gallup,News Chron,47.0,NaN,45.0,NaN,6.5,NaN,2.0,NaN
2,1952,Jan,NaN,NaN,NaN,Gallup,News Chron,44.5,-2.5,48.0,3.0,6.0,-0.5,-3.5,-5.5
3,1952,Feb,NaN,NaN,NaN,Gallup,News Chron,41.0,-3.5,47.0,-1.0,10.5,4.5,-6.0,-2.5
4,1952,Mar,NaN,NaN,NaN,Gallup,News Chron,41.5,0.5,48.0,1.0,9.5,-1.0,-6.5,-0.5
5,1952,Apr,NaN,NaN,NaN,Gallup,News Chron,43.5,2.0,49.0,1.0,NaN,NaN,-5.5,NaN
6,1952,May,NaN,NaN,NaN,Gallup,News Chron,43.5,0.0,49.0,0.0,7.0,NaN,-5.5,NaN
7,1952,Jun,NaN,NaN,NaN,Gallup,News Chron,40.5,-3.0,49.0,0.0,9.5,2.5,-8.5,-3.0
8,1952,Jul,NaN,NaN,NaN,Gallup,News Chron,40.0,-0.5,50.0,1.0,8.5,-1.0,-10.0,-1.5
9,1952,Aug,NaN,NaN,NaN,Gallup,News Chron,40.0,0.0,48.0,-2.0,10.5,2.0,-8.0,2.0


In [317]:
sheet_name = xl.sheet_names[5]
print(sheet_name)
df = xl.parse(sheet_name)
# df
df = df[['Year', 'Month', 'Fieldwork', 'Date', 'Polling', 'Publisher', 'Con',
       'Unnamed: 7', 'Lab', 'Unnamed: 9', 'LD', 'Unnamed: 11', 'Con lead',
       'Unnamed: 13']]
# 'Con good', 'Con bad',
#        'Con net', 'Lab good', 'Lab bad', 'Lab net', 'LD good', 'LD bad',
#        'LD net', 'Question', 'Govt good', 'Govt bad',
#        'Govt net', 'Question.1'
df.columns = ['Year', 'Month', 'Fieldwork', 'Publication Day', 'Polling', 'Publisher', 'Con',
       'Con Error', 'Lab', 'Lab Error', 'LD', 'LD Error', 'Con lead',
       'Con lead Error']
        

# drop empty rows
df.drop(df.index[df.isnull().all(axis=1)],inplace=True)

df["Year"] = df["Year"].fillna( method ='ffill',axis=0 ).astype('int')
df["Month"] = df["Month"].fillna( method ='ffill',axis=0 )

df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Start_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 0) ).astype("float").astype("Int64") )
df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Finish_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 1) ).astype("float").astype("Int64") )          
df.drop("Fieldwork",axis=1,inplace=True)


## Manual fixes("")

# 1959GE = 8 October 1959
df.loc[df["Polling"]=="Result",["Year","Month","Fieldwork_Start_Day","Fieldwork_Finish_Day","Publication Day"]]=\
    [1959,"Oct",8,8,8]

df["Publication Day"] = df["Publication Day"].astype("Int64")
df_list.append(df)
df        

55-59


,Year,Month,Fieldwork_Start_Day,Fieldwork_Finish_Day,Publication Day,Polling,Publisher,Con,Con Error,Lab,Lab Error,LD,LD Error,Con lead,Con lead Error
0,1955,Jun,NaN,NaN,NaN,Gallup,News Chron,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1955,Jul,2,13,NaN,Gallup,News Chron,47.0,NaN,43.0,NaN,9.0,NaN,4.0,NaN
2,1955,Aug,NaN,NaN,NaN,Gallup,News Chron,44.5,-2.5,47.5,4.5,7.0,-2.0,-3.0,-7.0
3,1955,Sep,9,19,NaN,Gallup,News Chron,48.0,3.5,44.0,-3.5,7.0,0.0,4.0,7.0
4,1955,Oct,19,26,NaN,Gallup,News Chron,46.5,-1.5,44.5,0.5,8.0,1.0,2.0,-2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,1959,Oct,3,6,NaN,Gallup,News Chron,46.5,1.5,47.5,0.5,5.5,-2.0,-1.0,1.0
77,1959,Oct,3,6,8,Gallup,News Chron,48.5,NaN,46.5,NaN,4.5,NaN,2.0,NaN
78,1959,Oct,2,5,8,NOP,DM,48.0,-0.4,44.1,-2.8,7.9,3.2,3.9,2.4
79,1959,Oct,NaN,NaN,8,NaN,DE,49.1,-0.6,45.4,-0.8,5.0,1.5,3.7,0.0


In [318]:
sheet_name = xl.sheet_names[6]
print(sheet_name)
df = xl.parse(sheet_name)
# df
df = df[['Year', 'Month', 'Fieldwork', 'Date', 'Polling', 'Publisher', 'Con',
       'Unnamed: 7', 'Lab', 'Unnamed: 9', 'LD', 'Unnamed: 11', 'Con lead',
       'Unnamed: 13']]
# 'Con good', 'Con bad',
#        'Con net', 'Lab good', 'Lab bad', 'Lab net', 'LD good', 'LD bad',
#        'LD net', 'Question', 'Govt good', 'Govt bad',
#        'Govt net', 'Question.1'
df.columns = ['Year', 'Month', 'Fieldwork', 'Publication Day', 'Polling', 'Publisher', 'Con',
       'Con Error', 'Lab', 'Lab Error', 'LD', 'LD Error', 'Con lead',
       'Con lead Error']
        

# drop empty rows
df.drop(df.index[df.isnull().all(axis=1)],inplace=True)

df["Year"] = df["Year"].fillna( method ='ffill',axis=0 ).astype('int')
df["Month"] = df["Month"].fillna( method ='ffill',axis=0 )

df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Start_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 0) ).astype("float").astype("Int64") )
df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Finish_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 1) ).astype("float").astype("Int64") )          
df.drop("Fieldwork",axis=1,inplace=True)


## Manual fixes("")

# 1964GE = 15 October 1964
df.loc[df["Polling"]=="Result",["Year","Month","Fieldwork_Start_Day","Fieldwork_Finish_Day","Publication Day"]]=\
    [1964,"Oct",15,15,15]

df["Publication Day"] = df["Publication Day"].astype("Int64")
df_list.append(df)
df        

59-64


,Year,Month,Fieldwork_Start_Day,Fieldwork_Finish_Day,Publication Day,Polling,Publisher,Con,Con Error,Lab,Lab Error,LD,LD Error,Con lead,Con lead Error
0,1959,Oct,NaN,NaN,NaN,Gallup,News Chron,48.0,NaN,46.0,NaN,5.0,NaN,2.0,NaN
1,1959,Nov,NaN,NaN,NaN,Gallup,News Chron,48.0,0.0,44.0,-2.0,7.0,2.0,4.0,2.0
2,1959,Dec,NaN,NaN,NaN,Gallup,News Chron,47.5,-0.5,44.0,0.0,7.5,0.5,3.5,-0.5
3,1960,Jan,1,6,NaN,Gallup,NaN,47.0,-0.5,43.5,-0.5,8.5,1.0,3.5,0.0
4,1960,Jan,9,20,NaN,Gallup,NaN,47.0,0.0,42.5,-1.0,9.5,1.0,4.5,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,1964,Oct,NaN,NaN,15,NaN,DE,44.5,-1.0,43.7,-0.9,11.1,1.7,0.8,-0.1
154,1964,Oct,9,13,15,NOP,DM,44.3,-1.9,47.4,1.1,7.9,0.8,-3.1,-3.0
155,1964,Oct,NaN,NaN,15,Gallup,DT,42.5,0.0,46.0,-2.5,11.0,2.5,-3.5,2.5
156,1964,Oct,NaN,NaN,15,Gallup,NaN,44.5,2.0,46.0,-4.5,9.0,2.5,-1.5,6.5


In [319]:
sheet_name = xl.sheet_names[7]
print(sheet_name)
df = xl.parse(sheet_name)
# df
df = df[['Year', 'Month', 'Fieldwork', 'Date', 'Polling', 'Publisher', 'Con',
       'Unnamed: 7', 'Lab', 'Unnamed: 9', 'LD', 'Unnamed: 11', 'Con lead',
       'Unnamed: 13']]
# 'Con good', 'Con bad',
#        'Con net', 'Lab good', 'Lab bad', 'Lab net', 'LD good', 'LD bad',
#        'LD net', 'Question', 'Govt good', 'Govt bad',
#        'Govt net', 'Question.1'
df.columns = ['Year', 'Month', 'Fieldwork', 'Publication Day', 'Polling', 'Publisher', 'Con',
       'Con Error', 'Lab', 'Lab Error', 'LD', 'LD Error', 'Con lead',
       'Con lead Error']
        

# drop empty rows
df.drop(df.index[df.isnull().all(axis=1)],inplace=True)

df["Year"] = df["Year"].fillna( method ='ffill',axis=0 ).astype('int')
df["Month"] = df["Month"].fillna( method ='ffill',axis=0 )

df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Start_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 0) ).astype("float").astype("Int64") )
df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Finish_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 1) ).astype("float").astype("Int64") )          
df.drop("Fieldwork",axis=1,inplace=True)


## Manual fixes("")

# 1966GE = 31 March 1966
df.loc[df["Polling"]=="Result",["Year","Month","Fieldwork_Start_Day","Fieldwork_Finish_Day","Publication Day"]]=\
    [1966,"Mar",31,31,31]

df["Publication Day"] = df["Publication Day"].astype("Int64")
df_list.append(df)
df        

64-66


,Year,Month,Fieldwork_Start_Day,Fieldwork_Finish_Day,Publication Day,Polling,Publisher,Con,Con Error,Lab,Lab Error,LD,LD Error,Con lead,Con lead Error
0,1964,Oct,22,25,NaN,NOP,DM,36.6,NaN,50.7,NaN,12.0,NaN,NaN,NaN
1,1964,Nov,5,8,NaN,NOP,DM,39.0,2.4,50.3,-0.4,10.5,-1.5,-11.3,NaN
2,1964,Nov,NaN,NaN,NaN,Gallup,DT,38.5,NaN,50.0,NaN,11.0,NaN,-11.5,NaN
3,1964,Nov,19,22,NaN,NOP,DM,37.9,-1.1,51.7,1.4,10.4,-0.1,-13.8,-2.5
4,1964,Dec,3,7,NaN,NOP,DM,38.8,0.9,48.7,-3.0,12.2,1.8,-9.9,3.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,1966,Mar,NaN,NaN,29,NaN,DE,40.0,1.0,52.5,-1.0,7.5,0.0,-12.5,2.0
63,1966,Mar,24,28,31,Gallup,DT,40.0,-2.0,51.0,0.5,8.0,1.0,-11.0,-2.5
64,1966,Mar,27,29,31,NOP,DM,41.6,2.0,50.6,-1.6,7.4,-0.2,-9.0,3.6
65,1966,Mar,NaN,NaN,31,NaN,DE,37.4,-2.6,54.1,1.6,7.7,0.2,-16.7,-4.2


In [320]:
sheet_name = xl.sheet_names[8]
print(sheet_name)
df = xl.parse(sheet_name).replace(" ",np.nan)
# df
df = df[['Year', 'Month', 'Fieldwork', 'Date', 'Polling', 'Publisher', 'Con',
       'Unnamed: 7', 'Lab', 'Unnamed: 9', 'LD', 'Unnamed: 11', 'Con lead',
       'Unnamed: 13']]
# 'Con good', 'Con bad',
#        'Con net', 'Lab good', 'Lab bad', 'Lab net', 'LD good', 'LD bad',
#        'LD net', 'Question', 'Govt good', 'Govt bad',
#        'Govt net', 'Question.1'
df.columns = ['Year', 'Month', 'Fieldwork', 'Publication Day', 'Polling', 'Publisher', 'Con',
       'Con Error', 'Lab', 'Lab Error', 'LD', 'LD Error', 'Con lead',
       'Con lead Error']
        

# drop empty rows
df.drop(df.index[df.isnull().all(axis=1)],inplace=True)

df["Year"] = df["Year"].fillna( method ='ffill',axis=0 ).astype('int')
df["Month"] = df["Month"].fillna( method ='ffill',axis=0 )

df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Start_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 0) ).astype("float").astype("Int64") )
df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Finish_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 1) ).astype("float").astype("Int64") )          
df.drop("Fieldwork",axis=1,inplace=True)


## Manual fixes("")

# 1970GE = 18 June 1970
df.loc[df["Polling"]=="Result",["Year","Month","Fieldwork_Start_Day","Fieldwork_Finish_Day","Publication Day"]]=\
    [1970,"Jun",18,18,18]

df["Publication Day"] = df["Publication Day"].astype("Int64")
df_list.append(df)
df        

66-70


,Year,Month,Fieldwork_Start_Day,Fieldwork_Finish_Day,Publication Day,Polling,Publisher,Con,Con Error,Lab,Lab Error,LD,LD Error,Con lead,Con lead Error
0,1966,Apr,14,19,NaN,Gallup,DT,35.5,NaN,54.5,NaN,8.5,NaN,-19.0,NaN
1,1966,Apr,21,15,NaN,NOP,DM,35.6,NaN,55.0,NaN,8.7,NaN,-19.4,NaN
2,1966,May,5,9,NaN,NOP,DM,34.1,-1.5,55.2,0.2,9.6,0.9,-21.1,-1.7
3,1966,May,6,10,NaN,Gallup,DT,35.5,0.0,53.5,3.2,10.0,3.2,-18.0,3.2
4,1966,Jun,3,7,NaN,Gallup,DT,39.5,4.0,52.0,-0.7,7.5,-0.7,-12.5,-0.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186,1970,Jun,12,16,18,NOP,DM,44.0,4.8,48.1,-3.5,6.4,-1.5,-4.1,8.3
187,1970,Jun,13,17,18,ORC,ES,46.5,4.5,45.5,-3.5,6.5,-1.5,1.0,8.0
188,1970,Jun,14,16,18,Gallup,DT,42.0,-3.5,49.0,1.0,7.5,2.0,-7.0,-4.5
189,1970,Jun,NaN,NaN,18,Harris,DE,46.0,1.0,48.0,2.0,5.0,-2.0,-2.0,-1.0


In [321]:
sheet_name = xl.sheet_names[9]
print(sheet_name)
df = xl.parse(sheet_name).replace(" ",np.nan)
# df
df = df[['Year', 'Month', 'Fieldwork', 'Date', 'Polling', 'Publisher', 'Con',
       'Unnamed: 7', 'Lab', 'Unnamed: 9', 'LD', 'Unnamed: 11', 'Con lead',
       'Unnamed: 13']]
# 'Con good', 'Con bad',
#        'Con net', 'Lab good', 'Lab bad', 'Lab net', 'LD good', 'LD bad',
#        'LD net', 'Question', 'Govt good', 'Govt bad',
#        'Govt net', 'Question.1'
df.columns = ['Year', 'Month', 'Fieldwork', 'Publication Day', 'Polling', 'Publisher', 'Con',
       'Con Error', 'Lab', 'Lab Error', 'LD', 'LD Error', 'Con lead',
       'Con lead Error']
        

# drop empty rows
df.drop(df.index[df.isnull().all(axis=1)],inplace=True)

df["Year"] = df["Year"].fillna( method ='ffill',axis=0 ).astype('int')
df["Month"] = df["Month"].fillna( method ='ffill',axis=0 )

df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Start_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 0) ).astype("float").astype("Int64") )
df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Finish_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 1) ).astype("float").astype("Int64") )          
df.drop("Fieldwork",axis=1,inplace=True)


## Manual fixes("")

# 1974GE = 28 February 1974
df.loc[df["Polling"]=="Result",["Year","Month","Fieldwork_Start_Day","Fieldwork_Finish_Day","Publication Day"]]=\
    [1974,"Feb",28,28,28]

df["Publication Day"] = df["Publication Day"].astype("Int64")
df_list.append(df)
df        

70-74


,Year,Month,Fieldwork_Start_Day,Fieldwork_Finish_Day,Publication Day,Polling,Publisher,Con,Con Error,Lab,Lab Error,LD,LD Error,Con lead,Con lead Error
0,1970,Jul,7,13,NaN,NOP,DM,51.2,NaN,40.4,NaN,6.5,NaN,10.8,NaN
1,1970,Jul,NaN,NaN,NaN,ORC,ES,51.0,NaN,40.0,NaN,8.0,NaN,11.0,NaN
2,1970,Aug,11,16,NaN,NOP,DM,47.3,-3.9,44.2,3.8,6.9,0.4,3.1,-7.7
3,1970,Aug,13,16,NaN,Gallup,DT,47.0,NaN,43.5,NaN,7.5,NaN,3.5,NaN
4,1970,Aug,NaN,NaN,NaN,ORC,ES,49.0,-2.0,41.0,1.0,8.0,0.0,8.0,-3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,1974,Feb,26,27,28,Gallup,DT,39.5,-2.5,37.5,-2.5,20.5,4.0,2.0,0.0
177,1974,Feb,27,27,28,ORC,ES,39.7,3.2,36.7,1.7,21.2,-4.3,3.0,1.5
178,1974,Feb,NaN,NaN,28,ORC,TT,38.7,-1.0,34.9,-1.8,23.0,1.8,3.8,2.8
179,1974,Feb,27,27,28,Marplan,Birmingham Evening Mail / LWT,36.5,-2.0,34.5,3.0,25.0,-3.0,2.0,-5.0


In [322]:
sheet_name = xl.sheet_names[10]
print(sheet_name)
df = xl.parse(sheet_name).replace(" ",np.nan)

# 1974GE2 = 10 October 1974
df.loc[df["Fieldwork"]=="11-16 or 17",["Fieldwork"]]= "11-16"

# df
df = df[['Year', 'Month', 'Fieldwork', 'Date', 'Polling', 'Publisher', 'Con',
       'Unnamed: 7', 'Lab', 'Unnamed: 9', 'LD', 'Unnamed: 11', 'Con lead',
       'Unnamed: 13']]
# 'Con good', 'Con bad',
#        'Con net', 'Lab good', 'Lab bad', 'Lab net', 'LD good', 'LD bad',
#        'LD net', 'Question', 'Govt good', 'Govt bad',
#        'Govt net', 'Question.1'
df.columns = ['Year', 'Month', 'Fieldwork', 'Publication Day', 'Polling', 'Publisher', 'Con',
       'Con Error', 'Lab', 'Lab Error', 'LD', 'LD Error', 'Con lead',
       'Con lead Error']
        

# drop empty rows
df.drop(df.index[df.isnull().all(axis=1)],inplace=True)

df["Year"] = df["Year"].fillna( method ='ffill',axis=0 ).astype('int')
df["Month"] = df["Month"].fillna( method ='ffill',axis=0 )

df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Start_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 0) ).astype("float").astype("Int64") )
df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Finish_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 1) ).astype("float").astype("Int64") )          
df.drop("Fieldwork",axis=1,inplace=True)


## Manual fixes("")

# 1974GE2 = 10 October 1974
df.loc[df["Polling"]=="Result",["Year","Month","Fieldwork_Start_Day","Fieldwork_Finish_Day","Publication Day"]]=\
    [1974,"Oct",10,10,10]

df["Publication Day"] = df["Publication Day"].astype("Int64")
df_list.append(df)
df        

74


,Year,Month,Fieldwork_Start_Day,Fieldwork_Finish_Day,Publication Day,Polling,Publisher,Con,Con Error,Lab,Lab Error,LD,LD Error,Con lead,Con lead Error
0,1974,Mar,12,17,NaN,NOP,NaN,35.7,NaN,43.3,NaN,17.9,NaN,-7.6,NaN
1,1974,Mar,13,18,NaN,Gallup,DT,35.0,NaN,43.0,NaN,19.0,NaN,-8.0,NaN
2,1974,Mar,19,24,NaN,NOP,NaN,33.6,-2.1,47.7,4.4,15.0,-2.9,-14.1,-6.5
3,1974,Mar,NaN,NaN,NaN,ORC,ES,35.0,NaN,44.0,NaN,17.0,NaN,-9.0,NaN
4,1974,Apr,2,7,NaN,NOP,NaN,36.4,2.8,45.1,-2.6,16.5,1.5,-8.7,5.4
5,1974,Apr,9,16,NaN,Gallup,DT,33.0,-2.0,49.0,6.0,15.5,-3.5,-16.0,-8.0
6,1974,Apr,NaN,NaN,NaN,ORC,ES,35.5,0.5,42.5,-1.5,18.0,1.0,-7.0,2.0
7,1974,May,8,13,NaN,Gallup,DT,33.0,0.0,46.5,-2.5,17.0,1.5,-13.5,2.5
8,1974,May,NaN,NaN,NaN,ORC,ES,35.0,-0.5,45.0,2.5,16.0,-2.0,-10.0,-3.0
9,1974,Jun,12,17,NaN,Gallup,DT,35.5,2.5,44.0,-2.5,17.0,0.0,-8.5,5.0


In [323]:
sheet_name = xl.sheet_names[11]
print(sheet_name)
df = xl.parse(sheet_name).replace(" ",np.nan)

# # 1974GE2 = 10 October 1974
# df.loc[df["Fieldwork"]=="11-16 or 17",["Fieldwork"]]= "11-16"

# df
df = df[['Year', 'Month', 'Fieldwork', 'Date', 'Polling', 'Publisher', 'Con',
       'Unnamed: 7', 'Lab', 'Unnamed: 9', 'LD', 'Unnamed: 11', 'Con lead',
       'Unnamed: 13']]
# 'Con good', 'Con bad',
#        'Con net', 'Lab good', 'Lab bad', 'Lab net', 'LD good', 'LD bad',
#        'LD net', 'Question', 'Govt good', 'Govt bad',
#        'Govt net', 'Question.1'
df.columns = ['Year', 'Month', 'Fieldwork', 'Publication Day', 'Polling', 'Publisher', 'Con',
       'Con Error', 'Lab', 'Lab Error', 'LD', 'LD Error', 'Con lead',
       'Con lead Error']
        

# drop empty rows
df.drop(df.index[df.isnull().all(axis=1)],inplace=True)

df["Year"] = df["Year"].fillna( method ='ffill',axis=0 ).astype('int')
df["Month"] = df["Month"].fillna( method ='ffill',axis=0 )

df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Start_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 0) ).astype("float").astype("Int64") )
df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Finish_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 1) ).astype("float").astype("Int64") )          
df.drop("Fieldwork",axis=1,inplace=True)


## Manual fixes("")

# 1979GE = 3 May 1979
df.loc[df["Polling"]=="Result",["Year","Month","Fieldwork_Start_Day","Fieldwork_Finish_Day","Publication Day"]]=\
    [1979,"May",3,3,3]

df["Publication Day"] = df["Publication Day"].astype("Int64")
df_list.append(df)
df        

74-79


,Year,Month,Fieldwork_Start_Day,Fieldwork_Finish_Day,Publication Day,Polling,Publisher,Con,Con Error,Lab,Lab Error,LD,LD Error,Con lead,Con lead Error
0,1974,Oct,22,27,NaN,NOP,DM,36.6,NaN,43.7,NaN,15.1,NaN,-7.1,NaN
1,1974,Oct,NaN,NaN,NaN,ORC,ES,NaN,NaN,NaN,NaN,NaN,NaN,-10.0,NaN
2,1974,Nov,14,18,NaN,Gallup,DT,35.0,NaN,46.5,NaN,14.5,NaN,-11.5,NaN
3,1974,Nov,26,1,NaN,NOP,DM,38.4,1.8,47.7,4.0,11.1,-4.0,-9.3,-2.2
4,1974,Nov,NaN,NaN,NaN,ORC,ES,NaN,NaN,NaN,NaN,NaN,NaN,2.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,1979,May,1,2,3,Gallup,DT,43.0,-5,41.0,1.0,13.5,3.0,2.0,-6.0
195,1979,May,2,2,3,MORI,ES,45.0,1,37.0,-4.0,15.0,3.0,8.0,5.0
197,1979,May,2,3,4,Gallup,BBC,45.6,2.6,37.9,-3.1,13.5,0.0,7.7,5.7
198,1979,May,3,3,3,ORC,ITN,44.0,NaN,38.0,NaN,14.0,NaN,6.0,NaN


In [324]:
sheet_name = xl.sheet_names[12]
print(sheet_name)
df = xl.parse(sheet_name).replace(" ",np.nan)

## Manual Fix
df.loc[df["Fieldwork"]=="28-2 or 11-16",["Fieldwork"]]= "28-2"
df.loc[df["Fieldwork"]=="11-16 or 5-10",["Fieldwork"]]= "11-16"
df.loc[df["Date"]=="8-9",["Date"]]= "8"



# df
df = df[['Year', 'Month', 'Fieldwork', 'Date', 'Polling', 'Publisher', 'Con',
       'Unnamed: 7', 'Lab', 'Unnamed: 9', 'LD', 'Unnamed: 11', 'Con lead',
       'Unnamed: 13']]
# 'Con good', 'Con bad',
#        'Con net', 'Lab good', 'Lab bad', 'Lab net', 'LD good', 'LD bad',
#        'LD net', 'Question', 'Govt good', 'Govt bad',
#        'Govt net', 'Question.1'
df.columns = ['Year', 'Month', 'Fieldwork', 'Publication Day', 'Polling', 'Publisher', 'Con',
       'Con Error', 'Lab', 'Lab Error', 'LD', 'LD Error', 'Con lead',
       'Con lead Error']
        

# drop empty rows
df.drop(df.index[df.isnull().all(axis=1)],inplace=True)

df["Year"] = df["Year"].fillna( method ='ffill',axis=0 ).astype('int')
df["Month"] = df["Month"].fillna( method ='ffill',axis=0 )

df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Start_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 0) ).astype("float").astype("Int64") )
df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Finish_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 1) ).astype("float").astype("Int64") )          
df.drop("Fieldwork",axis=1,inplace=True)


## Manual fixes("")

# 1983GE = 9 June 1983
df.loc[df["Polling"]=="Result",["Year","Month","Fieldwork_Start_Day","Fieldwork_Finish_Day","Publication Day"]]=\
    [1983,"Jun",9,9,9]

df["Publication Day"] = df["Publication Day"].astype("float").astype("Int64")
df_list.append(df)
df        

79-83


,Year,Month,Fieldwork_Start_Day,Fieldwork_Finish_Day,Publication Day,Polling,Publisher,Con,Con Error,Lab,Lab Error,LD,LD Error,Con lead,Con lead Error
0,1979,Jun,NaN,7,NaN,NOP,NaN,44.2,NaN,45.5,NaN,8.2,NaN,-1.3,NaN
1,1979,Jun,13,18,NaN,Gallup,DT,42.0,NaN,43.5,NaN,12.0,NaN,-1.5,NaN
2,1979,Jul,NaN,5,NaN,NOP,NaN,43.1,-1.1,45.4,-0.1,9.5,1.3,-2.3,-1.0
3,1979,Jul,11,16,NaN,Gallup,DT,41.0,-1.0,46.0,2.5,11.5,-0.5,-5.0,-3.5
4,1979,Aug,8,13,NaN,Gallup,DT,41.5,0.5,44.0,-2.0,12.5,1.0,-2.5,2.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214,1983,Jun,8,8,9,NOP,Northcliffe,47.0,1.0,25.0,-3.0,26.0,2.0,22.0,4.0
215,1983,Jun,8,8,9,MORI,ES,44.0,-3.0,28.0,2.0,26.0,1.0,16.0,-5.0
217,1983,Jun,NaN,NaN,9,Harris,ITN,42.0,NaN,28.0,NaN,26.0,NaN,14.0,NaN
218,1983,Jun,NaN,NaN,8,Gallup,BBC,43.9,NaN,28.7,NaN,25.8,NaN,15.2,NaN


In [325]:
sheet_name = xl.sheet_names[13]
print(sheet_name)
df = xl.parse(sheet_name).replace(" ",np.nan)

## Manual Fix
# df.loc[df["Fieldwork"]=="28-2 or 11-16",["Fieldwork"]]= "28-2"
# df.loc[df["Fieldwork"]=="11-16 or 5-10",["Fieldwork"]]= "11-16"
# df.loc[df["Date"]=="8-9",["Date"]]= "8"



# df
df = df[['Year', 'Month', 'Fieldwork', 'Date', 'Polling', 'Publisher', 'Con',
       'Unnamed: 7', 'Lab', 'Unnamed: 9', 'LD', 'Unnamed: 11', 'SDP',
       'Unnamed: 13', 'Green', 'Unnamed: 15', 'Con lead', 'Unnamed: 17',]]
# 'Con good', 'Con bad',
#        'Con net', 'Lab good', 'Lab bad', 'Lab net', 'LD good', 'LD bad',
#        'LD net', 'Question', 'Govt good', 'Govt bad',
#        'Govt net', 'Question.1'
df.columns = ['Year', 'Month', 'Fieldwork', 'Publication Day', 'Polling', 'Publisher', 'Con',
       'Con Error', 'Lab', 'Lab Error', 'LD', 'LD Error', 'SDP', 'SDP Error', 'Green','Green Error', 'Con lead',
       'Con lead Error']
        

# drop empty rows
df.drop(df.index[df.isnull().all(axis=1)],inplace=True)

df["Year"] = df["Year"].fillna( method ='ffill',axis=0 ).astype('int')
df["Month"] = df["Month"].fillna( method ='ffill',axis=0 )

df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Start_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 0) ).astype("float").astype("Int64") )
df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Finish_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 1) ).astype("float").astype("Int64") )          
df.drop("Fieldwork",axis=1,inplace=True)


## Manual fixes("")

# 1987GE = 11 June 1987
df.loc[df["Polling"]=="Result",["Year","Month","Fieldwork_Start_Day","Fieldwork_Finish_Day","Publication Day"]]=\
    [1987,"Jun",11,11,11]

df["Publication Day"] = df["Publication Day"].astype("float").astype("Int64")
df_list.append(df)
df        

83-87


,Year,Month,Fieldwork_Start_Day,Fieldwork_Finish_Day,Publication Day,Polling,Publisher,Con,Con Error,Lab,Lab Error,LD,LD Error,SDP,SDP Error,Green,Green Error,Con lead,Con lead Error
0,1983,Jun,16,20,4,MORI,DS,43.0,NaN,27.0,NaN,29.0,NaN,NaN,NaN,NaN,NaN,16.0,NaN
1,1983,Jul,7,11,13,Gallup,DT,44.0,NaN,28.5,NaN,26.0,NaN,NaN,NaN,NaN,NaN,15.5,NaN
2,1983,Jul,NaN,12,NaN,NOP,NaN,46.4,NaN,27.4,NaN,24.3,NaN,NaN,NaN,NaN,NaN,19.0,NaN
3,1983,Jul,21,25,1,MORI,DE,44.0,1.0,30.0,3.0,25.0,-4.0,NaN,NaN,NaN,NaN,14.0,-2.0
4,1983,Aug,NaN,2,NaN,NOP,NaN,47.1,0.7,24.5,-2.9,27.1,2.8,NaN,NaN,NaN,NaN,22.6,3.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
313,1987,Jun,9,10,11,MORI,TT,44.0,0.0,32.0,1.0,22.0,-2.0,NaN,NaN,NaN,NaN,12.0,-1.0
314,1987,Jun,10,10,NaN,NOP,Ind,42.0,1.0,35.0,2.0,21.0,-2.0,NaN,NaN,NaN,NaN,7.0,-1.0
316,1987,Jun,NaN,NaN,NaN,NaN,ITN,41.8,NaN,32.0,NaN,23.4,NaN,NaN,NaN,NaN,NaN,9.8,NaN
317,1987,Jun,NaN,NaN,NaN,NaN,BBC,40.4,NaN,35.4,NaN,22.5,NaN,NaN,NaN,NaN,NaN,5.0,NaN


In [326]:
sheet_name = xl.sheet_names[14]
print(sheet_name)
df = xl.parse(sheet_name).replace(" ",np.nan)

## Manual Fix
df.loc[df["Fieldwork"]=="24-29?",["Fieldwork"]]= "24-29"
df.loc[df["Fieldwork"]=="29?",["Fieldwork"]]= "29"
df.loc[df["Fieldwork"]=="2-4?",["Fieldwork"]]= "2-4"
df.loc[df["Fieldwork"]=="10/12",["Fieldwork"]]= "10"
df.loc[df["Date"]=='2?',["Date"]]= '2'
df.loc[df["Date"]=='29?',["Date"]]= '29'


# df.loc[df["Fieldwork"]=="11-16 or 5-10",["Fieldwork"]]= "11-16"
# df.loc[df["Date"]=="8-9",["Date"]]= "8"



# df
df = df[['Year', 'Month', 'Fieldwork', 'Date', 'Polling', 'Publisher', 'Con',
       'Unnamed: 7', 'Lab', 'Unnamed: 9', 'LD', 'Unnamed: 11', 'SDP',
       'Unnamed: 13', 'Green', 'Unnamed: 15', 'Con lead', 'Unnamed: 17',]]
# 'Con good', 'Con bad',
#        'Con net', 'Lab good', 'Lab bad', 'Lab net', 'LD good', 'LD bad',
#        'LD net', 'Question', 'Govt good', 'Govt bad',
#        'Govt net', 'Question.1'
df.columns = ['Year', 'Month', 'Fieldwork', 'Publication Day', 'Polling', 'Publisher', 'Con',
       'Con Error', 'Lab', 'Lab Error', 'LD', 'LD Error', 'SDP', 'SDP Error', 'Green','Green Error', 'Con lead',
       'Con lead Error']
        

# drop empty rows
df.drop(df.index[df.isnull().all(axis=1)],inplace=True)

df["Year"] = df["Year"].fillna( method ='ffill',axis=0 ).astype('int')
df["Month"] = df["Month"].fillna( method ='ffill',axis=0 )

df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Start_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 0) ).astype("float").astype("Int64") )
df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Finish_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 1) ).astype("float").astype("Int64") )          
df.drop("Fieldwork",axis=1,inplace=True)


## Manual fixes("")

# 1992GE = 9 April 1992
df.loc[df["Polling"]=="Result",["Year","Month","Fieldwork_Start_Day","Fieldwork_Finish_Day","Publication Day"]]=\
    [1992,"Apr",9,9,9]

df["Publication Day"] = df["Publication Day"].astype("float").astype("Int64")
df_list.append(df)
df        

87-92


,Year,Month,Fieldwork_Start_Day,Fieldwork_Finish_Day,Publication Day,Polling,Publisher,Con,Con Error,Lab,Lab Error,LD,LD Error,SDP,SDP Error,Green,Green Error,Con lead,Con lead Error
0,1987,Jun,24,29,29,NOP,NaN,42.0,NaN,33.0,NaN,23.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN
1,1987,Jun,25,1,5,MORI,ST,49.0,NaN,31.0,NaN,17.0,NaN,NaN,NaN,NaN,NaN,18.0,7.0
2,1987,Jul,NaN,6,NaN,NOP,NaN,49.0,7.0,31.0,-2.0,18.0,-5.0,NaN,NaN,NaN,NaN,18.0,9.0
3,1987,Jul,8,13,19,Gallup,DT,44.5,NaN,33.0,NaN,20.5,NaN,NaN,NaN,NaN,NaN,11.5,4.5
4,1987,Jul,16,16,19,Harris,OB,48.0,NaN,33.0,NaN,17.0,NaN,NaN,NaN,NaN,NaN,15.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
560,1992,Apr,8,8,9,ICM,GU,38.0,1.0,38.0,-1.0,20.0,1.0,NaN,NaN,NaN,NaN,0.0,2.0
562,1992,Apr,NaN,NaN,NaN,ICM,Sun,38.0,NaN,41.0,NaN,18.0,NaN,NaN,NaN,NaN,NaN,-3.0,NaN
563,1992,Apr,NaN,NaN,NaN,Harris,ITN,41.1,NaN,36.3,NaN,18.0,NaN,NaN,NaN,NaN,NaN,4.8,NaN
564,1992,Apr,NaN,NaN,NaN,NOP,BBC,40.0,NaN,36.3,NaN,18.3,NaN,NaN,NaN,NaN,NaN,3.7,NaN


In [327]:
sheet_name = xl.sheet_names[15]
print(sheet_name)
df = xl.parse(sheet_name).replace(" ",np.nan)

## Manual Fix

df.loc[347:401,"Year"]=np.nan #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

df.loc[df["Fieldwork"]=='25-28?',["Fieldwork"]]= '25-28'
# df.loc[df["Fieldwork"]=="29?",["Fieldwork"]]= "29"
# df.loc[df["Fieldwork"]=="2-4?",["Fieldwork"]]= "2-4"
# df.loc[df["Fieldwork"]=="10/12",["Fieldwork"]]= "10"
df.loc[df["Date"]=='7?',["Date"]]= '7'
df.loc[df["Date"]=='19??',["Date"]]= '19'


# df.loc[df["Fieldwork"]=="11-16 or 5-10",["Fieldwork"]]= "11-16"
# df.loc[df["Date"]=="8-9",["Date"]]= "8"



# # df
df = df[['Year', 'Month', 'Fieldwork', 'Date', 'Polling', 'Publisher', 'Con',
       'Unnamed: 7', 'Lab', 'Unnamed: 9', 'LD', 'Unnamed: 11', 'Referendum',
       'Unnamed: 13', 'Con lead', 'Unnamed: 15']]
# # 'Con good', 'Con bad',
# #        'Con net', 'Lab good', 'Lab bad', 'Lab net', 'LD good', 'LD bad',
# #        'LD net', 'Question', 'Govt good', 'Govt bad',
# #        'Govt net', 'Question.1'
df.columns = ['Year', 'Month', 'Fieldwork', 'Publication Day', 'Polling', 'Publisher', 'Con',
       'Con Error', 'Lab', 'Lab Error', 'LD', 'LD Error',  'Referendum','Referendum Error', 'Con lead',
       'Con lead Error']
        

# drop empty rows
df.drop(df.index[df.isnull().all(axis=1)],inplace=True)

df["Year"] = df["Year"].fillna( method ='ffill',axis=0 ).astype('int')
df["Month"] = df["Month"].fillna( method ='ffill',axis=0 )

df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Start_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 0) ).astype("float").astype("Int64") )
df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Finish_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 1) ).astype("float").astype("Int64") )          
df.drop("Fieldwork",axis=1,inplace=True)


## Manual fixes("")

# 1997GE = 1 May 1997
df.loc[df["Polling"]=="Result",["Year","Month","Fieldwork_Start_Day","Fieldwork_Finish_Day","Publication Day"]]=\
    [1997,"May",1,1,1]

df["Publication Day"] = df["Publication Day"].astype("float").astype("Int64")
df_list.append(df)
df        

92-97


,Year,Month,Fieldwork_Start_Day,Fieldwork_Finish_Day,Publication Day,Polling,Publisher,Con,Con Error,Lab,Lab Error,LD,LD Error,Referendum,Referendum Error,Con lead,Con lead Error
0,1992,Apr,10,11,NaN,ASL,NaN,42.0,NaN,36.0,NaN,18.0,NaN,NaN,NaN,6.0,NaN
1,1992,Apr,10,12,NaN,Gallup,NaN,39.0,NaN,38.0,NaN,19.0,NaN,NaN,NaN,1.0,NaN
2,1992,Apr,25,28,3,MORI,ST,43.0,NaN,38.0,NaN,16.0,NaN,NaN,NaN,5.0,NaN
3,1992,Apr,15,16,19,Harris,OB,44.0,NaN,36.0,NaN,17.0,NaN,NaN,NaN,8.0,NaN
4,1992,Apr,25,28,NaN,MORI,NaN,39.0,-4.0,37.0,-1.0,18.0,2.0,NaN,NaN,2.0,-3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401,1997,Apr,30,30,1,MORI,ES,29.0,NaN,46.0,NaN,19.0,NaN,NaN,NaN,-17.0,3.0
403,1997,Apr,NaN,NaN,NaN,G9000,NaN,31.0,1.2,50.0,-3.7,12.6,1.8,NaN,NaN,-19.0,4.9
405,1997,Apr,NaN,NaN,NaN,MORI,ITN,30.0,NaN,46.0,NaN,18.0,NaN,NaN,NaN,-16.0,NaN
406,1997,Apr,NaN,NaN,NaN,NOP,BBC,29.0,NaN,47.0,NaN,17.0,NaN,NaN,NaN,-18.0,NaN


In [328]:
sheet_name = xl.sheet_names[16]
print(sheet_name)
df = xl.parse(sheet_name).replace(" ",np.nan)

## Manual Fix

# df.loc[347:401,"Year"]=np.nan ####!

df.loc[df["Date"]=='19?',["Date"]]= '19'
# df.loc[df["Fieldwork"]=="29?",["Fieldwork"]]= "29"
# df.loc[df["Fieldwork"]=="2-4?",["Fieldwork"]]= "2-4"
# df.loc[df["Fieldwork"]=="10/12",["Fieldwork"]]= "10"
# df.loc[df["Date"]=='7?',["Date"]]= '7'
# df.loc[df["Date"]=='19??',["Date"]]= '19'


# df.loc[df["Fieldwork"]=="11-16 or 5-10",["Fieldwork"]]= "11-16"
# df.loc[df["Date"]=="8-9",["Date"]]= "8"



# # df
df = df[['Year', 'Month', 'Fieldwork', 'Date', 'Polling', 'Publisher', 'Con',
       'Unnamed: 7', 'Lab', 'Unnamed: 9', 'LD', 'Unnamed: 11',  'Con lead', 'Unnamed: 13']]
# # 'Con good', 'Con bad',
# #        'Con net', 'Lab good', 'Lab bad', 'Lab net', 'LD good', 'LD bad',
# #        'LD net', 'Question', 'Govt good', 'Govt bad',
# #        'Govt net', 'Question.1'
df.columns = ['Year', 'Month', 'Fieldwork', 'Publication Day', 'Polling', 'Publisher', 'Con',
       'Con Error', 'Lab', 'Lab Error', 'LD', 'LD Error',  'Con lead',
       'Con lead Error']
        

# drop empty rows
df.drop(df.index[df.isnull().all(axis=1)],inplace=True)

df["Year"] = df["Year"].fillna( method ='ffill',axis=0 ).astype('int')
df["Month"] = df["Month"].fillna( method ='ffill',axis=0 )

df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Start_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 0) ).astype("float").astype("Int64") )
df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Finish_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 1) ).astype("float").astype("Int64") )          
df.drop("Fieldwork",axis=1,inplace=True)


## Manual fixes("")

# 2001GE = 7 June 2001
df.loc[df["Polling"]=="Result",["Year","Month","Fieldwork_Start_Day","Fieldwork_Finish_Day","Publication Day"]]=\
    [2001,"Jun",7,7,7]

df["Publication Day"] = df["Publication Day"].astype("float").astype("Int64")
df_list.append(df)
df        

97-01


,Year,Month,Fieldwork_Start_Day,Fieldwork_Finish_Day,Publication Day,Polling,Publisher,Con,Con Error,Lab,Lab Error,LD,LD Error,Con lead,Con lead Error
0,1997,May,NaN,NaN,NaN,G9000,DT,23.3,-7.7,56.9,6.9,14.7,2.1,-33.6,-14.6
1,1997,May,29,4,5,Gallup,DT,23.0,-10.0,59.0,13.0,13.0,-3.0,-36.0,-23.0
2,1997,Jun,6,7,10,ICM,GU,23.0,-10.0,62.0,19.0,14.0,-4.0,-39.0,-29.0
3,1997,Jun,27,28,1,ICM,GU,23.0,0.0,61.0,-1.0,12.0,-2.0,-38.0,1.0
4,1997,Jun,20,23,26,MORI,TT,24.0,-5.0,58.0,12.0,15.0,-4.0,-34.0,-17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,2001,Jun,5,6,7,MORI,TT,30.0,0.0,45.0,-3.0,18.0,2.0,-15.0,3.0
280,2001,Jun,6,6,7,Gallup,DT,32.0,1.0,48.0,1.0,18.0,2.0,-16.0,0.0
282,2001,Jun,NaN,NaN,NaN,BBC,NOP,32.0,NaN,44.0,NaN,17.0,NaN,-12.0,NaN
283,2001,Jun,NaN,NaN,NaN,ITN,MORI,31.0,NaN,44.0,NaN,19.0,NaN,-13.0,NaN


In [329]:
sheet_name = xl.sheet_names[17]
print(sheet_name)
df = xl.parse(sheet_name).replace(" ",np.nan)

## Manual Fix

##### leadership ratings order isn't clear!
# assume same as others?
# Con good	Con bad	Con net	Lab good	Lab bad	Lab net	LD good	LD bad	LD net->question!


df.loc[df["Fieldwork"]=='8-10?',["Fieldwork"]]= '8-10'
df.loc[df["Published"]=="2-3",["Published"]]= "2"
df.loc[df["Published"]=="3-4",["Published"]]= "3"

# df.loc[df["Fieldwork"]=="2-4?",["Fieldwork"]]= "2-4"
# df.loc[df["Fieldwork"]=="10/12",["Fieldwork"]]= "10"
# df.loc[df["Date"]=='7?',["Date"]]= '7'
# df.loc[df["Date"]=='19??',["Date"]]= '19'


# df.loc[df["Fieldwork"]=="11-16 or 5-10",["Fieldwork"]]= "11-16"
# df.loc[df["Date"]=="8-9",["Date"]]= "8"



# # # df
df = df[['Year', 'Month', 'Fieldwork', 'Published', 'Polling', 'Publisher',
       'Con', 'Unnamed: 7', 'Lab', 'Unnamed: 9', 'LD', 'Unnamed: 11',
       'Con lead', 'Unnamed: 13',
       'Party leader rating', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24',
       'Unnamed: 25', 'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28',
       'Unnamed: 29', 'Unnamed: 30']]
# # # 'Con good', 'Con bad',
# # #        'Con net', 'Lab good', 'Lab bad', 'Lab net', 'LD good', 'LD bad',
# # #        'LD net', 'Question', 'Govt good', 'Govt bad',
# # #        'Govt net', 'Question.1'
df.columns = ['Year', 'Month', 'Fieldwork', 'Publication Day', 'Polling', 'Publisher', 'Con',
       'Con Error', 'Lab', 'Lab Error', 'LD', 'LD Error',  'Con lead',
       'Con lead Error',
        "Leader: Con good","Leader: Con bad","Leader: Con net",
        "Leader: Lab good","Leader: Lab bad","Leader: Lab net",
        "Leader: LD good", "Leader: LD bad", "Leader: LD net",
        "Leader: Question",
             ]
        # Con good	Con bad	Con net	Lab good	Lab bad	Lab net	LD good	LD bad	LD net->question!

# drop empty rows
df.drop(df.index[df.isnull().all(axis=1)],inplace=True)

df["Year"] = df["Year"].fillna( method ='ffill',axis=0 ).astype('int')
df["Month"] = df["Month"].fillna( method ='ffill',axis=0 )

df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Start_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 0) ).astype("float").astype("Int64") )
df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Finish_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 1) ).astype("float").astype("Int64") )          
df.drop("Fieldwork",axis=1,inplace=True)


## Manual fixes("")

# 2005GE = 5 May 2005
df.loc[df["Polling"]=="Result",["Year","Month","Fieldwork_Start_Day","Fieldwork_Finish_Day","Publication Day"]]=\
    [2005,"May",5,5,5]
df["Publication Day"] = df["Publication Day"].apply(lambda x: x.split("/")[0] if pd.notnull(x) and not isinstance(x,int) else x)

df["Publication Day"] = df["Publication Day"].astype("float").astype("Int64")
df_list.append(df)
df        

01-05


,Year,Month,Fieldwork_Start_Day,Fieldwork_Finish_Day,Publication Day,Polling,Publisher,Con,Con Error,Lab,...,Leader: Con good,Leader: Con bad,Leader: Con net,Leader: Lab good,Leader: Lab bad,Leader: Lab net,Leader: LD good,Leader: LD bad,Leader: LD net,Leader: Question
0,2001,Jun,21,26,NaN,MORI,NaN,25.0,NaN,49.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2001,Jul,13,14,NaN,ICM,GU,30.0,NaN,46.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2001,Jul,19,23,26,MORI,TT,25.0,0.0,52.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2001,Jul,20,5,6,ICM,R4,29.0,-1.0,53.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2001,Aug,9,14,10,MORI,GMB,23.0,-2.0,53.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308,2005,May,2,3,5,Populus,TT,32.0,5.0,38.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
309,2005,May,3,4,5,YouGov,DT,32.0,0.0,37.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
310,2005,May,3,4,5,MORI,ES,33.0,4.0,38.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
312,2005,May,NaN,NaN,NaN,NaN,NaN,33.0,NaN,37.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [330]:
sheet_name = xl.sheet_names[18]
print(sheet_name)
df = xl.parse(sheet_name).replace(" ",np.nan)

## Manual Fix

##### leadership ratings order isn't clear!
# assume same as others?
# Con good	Con bad	Con net	Lab good	Lab bad	Lab net	LD good	LD bad	LD net->question!


df.loc[df["Fieldwork"]=='15 ?',["Fieldwork"]]= '15'
df.loc[df["Fieldwork"]=="29-1 ?",["Fieldwork"]]= "29-1"
# df.loc[df["Published"]=="3-4",["Published"]]= "3"

df.loc[df["Fieldwork"]=="13-14/11",["Fieldwork"]]= "13-14"
# df.loc[df["Fieldwork"]=="10/12",["Fieldwork"]]= "10"
# df.loc[df["Date"]=='7?',["Date"]]= '7'
# df.loc[df["Date"]=='19??',["Date"]]= '19'


# df.loc[df["Fieldwork"]=="11-16 or 5-10",["Fieldwork"]]= "11-16"
# df.loc[df["Date"]=="8-9",["Date"]]= "8"



# # # # df
df = df[['Year', 'Month', 'Fieldwork', 'Unnamed: 3', 'Published', 'Polling',
       'Publisher', 'Con', 'Unnamed: 8', 'M', 'F', 'Net', 'Lab', 'Unnamed: 13',
       'M.1', 'F.1', 'Net.1', 'LD', 'Unnamed: 18', 'M.2', 'F.2', 'Net.2',
       'Con lead', 'Unnamed: 23', 'Con good',
       'Con bad', 'Con net', 'Lab good', 'Lab bad', 'Lab net', 'LD good',
       'LD bad', 'LD net', 'Question', ]]
# # # # 'Con good', 'Con bad',
# # # #        'Con net', 'Lab good', 'Lab bad', 'Lab net', 'LD good', 'LD bad',
# # # #        'LD net', 'Question', 'Govt good', 'Govt bad',
# # # #        'Govt net', 'Question.1'
df.columns = ['Year', 'Month', 'Fieldwork', 'Unnamed: 3', 'Publication Day', 'Polling', 'Publisher',
              'Con', 'Con Error', 'Con Male', 'Con Female', 'Con Male-Female',
              'Lab', 'Lab Error', 'Lab Male', 'Lab Female', 'Lab Male-Female',
              'LD', 'LD Error', 'LD Male', 'LD Female', 'LD Male-Female',
              'Con lead', 'Con lead Error',
        "Leader: Con good","Leader: Con bad","Leader: Con net",
        "Leader: Lab good","Leader: Lab bad","Leader: Lab net",
        "Leader: LD good", "Leader: LD bad", "Leader: LD net",
        "Leader: Question",
             ]
#         # Con good	Con bad	Con net	Lab good	Lab bad	Lab net	LD good	LD bad	LD net->question!

# drop empty rows
df.drop(df.index[df.isnull().all(axis=1)],inplace=True)

df["Year"] = df["Year"].fillna( method ='ffill',axis=0 ).astype('int')
df["Month"] = df["Month"].fillna( method ='ffill',axis=0 )

df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Start_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 0) ).astype("float").astype("Int64") )
df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Finish_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 1) ).astype("float").astype("Int64") )          
df.drop("Fieldwork",axis=1,inplace=True)

df["Fieldwork_Start_Day"] = df["Unnamed: 3"].apply(lambda x: x.day)
df.drop("Unnamed: 3",axis=1,inplace=True)

# ## Manual fixes("")

# 2010GE = 6 May 2010
df.loc[df["Polling"]=="Result",["Year","Month","Fieldwork_Start_Day","Fieldwork_Finish_Day","Publication Day"]]=\
    [2010,"May",6,6,6]
# df["Publication Day"] = df["Publication Day"].apply(lambda x: x.split("/")[0] if pd.notnull(x) and not isinstance(x,int) else x)

df["Publication Day"] = df["Publication Day"].astype("float").astype("Int64")
df_list.append(df)
df        

05-10


,Year,Month,Fieldwork_Start_Day,Fieldwork_Finish_Day,Publication Day,Polling,Publisher,Con,Con Error,Con Male,...,Leader: Con good,Leader: Con bad,Leader: Con net,Leader: Lab good,Leader: Lab bad,Leader: Lab net,Leader: LD good,Leader: LD bad,Leader: LD net,Leader: Question
0,2005,May,19.0,23,NaN,MORI,First Tuesday,27.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2005,May,19.0,23,NaN,MORI,FT,30.0,NaN,NaN,...,28.0,48.0,-20.0,39.0,55.0,-16.0,49.0,28.0,21.0,Are you satisfied or dissatisfied with the way...
2,2005,May,24.0,26,NaN,YouGov,DT,31.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2005,Jun,16.0,20,7,MORI,Social Research Institute,29.0,-1.0,NaN,...,23.0,49.0,-26.0,39.0,52.0,-13.0,46.0,26.0,20.0,Are you satisfied or dissatisfied with the way...
4,2005,Jun,17.0,19,21,ICM,GU,31.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688,2,May,4.0,5,6,YouGov,Sun,35.0,0.0,33.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
689,2,May,4.0,5,6,Opinium,Express,35.0,2.0,35.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
690,2,May,4.0,5,6,ComRes,ITV/Ind,37.0,0.0,39.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
691,1,May,5.0,5,6,MORI,ES,36.0,0.0,34.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [331]:
sheet_name = xl.sheet_names[19]
print(sheet_name)
df = xl.parse(sheet_name).replace(" ",np.nan)
import datetime
## Manual Fix

##### leadership ratings order isn't clear!
# assume same as others?
# Con good	Con bad	Con net	Lab good	Lab bad	Lab net	LD good	LD bad	LD net->question!


# Ashcroft - did the fieldwork really take over a month?
df.loc[df["Fieldwork"]=='1-5 Sept',["Fieldwork"]]= '1-5'
# df.loc[df["Fieldwork"]=="29-1 ?",["Fieldwork"]]= "29-1"
df.loc[df["Published"]=="20?",["Published"]]= "20"

df.loc[df["Published"]=="23/24",["Published"]]= "23"
df.loc[df["Published"]==datetime.datetime(2011, 9, 25, 0, 0),["Published"]]= "25"
df.loc[df["Published"]==datetime.datetime(2012, 11, 21, 0, 0),["Published"]]= "21"
df.loc[df["Published"]==datetime.datetime(2014, 1, 4, 0, 0),["Published"]]= "4"
df.loc[df["Published"]==datetime.datetime(2013, 12, 26, 0, 0),["Published"]]= "26"

# # df.loc[df["Fieldwork"]=="10/12",["Fieldwork"]]= "10"
# # df.loc[df["Date"]=='7?',["Date"]]= '7'
# # df.loc[df["Date"]=='19??',["Date"]]= '19'


# # df.loc[df["Fieldwork"]=="11-16 or 5-10",["Fieldwork"]]= "11-16"
# # df.loc[df["Date"]=="8-9",["Date"]]= "8"



# # # # df
df = df[['Year', 'Month', 'Fieldwork', 'Unnamed: 3', 'Published', 
       'Polling', 'Publisher', 'Con', 'Unnamed: 9', 'Lab', 'Unnamed: 11', 'LD',
       'Unnamed: 13', 'Con lead', 'Unnamed: 15', 
       'UKIP', 'Green', 'BNP', 'Con good', 'Con bad',
       'Con net', 'Lab good', 'Lab bad', 'Lab net', 'LD good', 'LD bad',
       'LD net', 'UKIP good', 'UKIP bad', 'UKIP net', 'Grn good', 'Grn bad',
       'Grn net', 'Question', 'Con Best PM', 'Lab Best PM', 'LD Best PM',
       'UKIP Best PM', 'Govt good', 'Govt bad', 'Govt net',
       'Question.1', 'Con.1', 'Lab.1', 'LD.1', 'UKIP.1', 'Unnamed: 52']]

df.columns = ['Year', 'Month', 'Fieldwork', 'Unnamed: 3', 'Publication Day', 'Polling', 'Publisher',
              'Con', 'Con Error',
              'Lab', 'Lab Error',
              'LD', 'LD Error',
              'Con lead', 'Con lead Error',
              'UKIP', 'Green', 'BNP',
            "Leader: Con good","Leader: Con bad","Leader: Con net",
            "Leader: Lab good","Leader: Lab bad","Leader: Lab net",
            "Leader: LD good", "Leader: LD bad", "Leader: LD net",
            "Leader: UKIP good","Leader: UKIP bad","Leader: UKIP net",
            "Leader: Grn good","Leader: Grn bad","Leader: Grn net",
            "Leader: Question",
            "Con Best PM","Lab Best PM","LD Best PM","UKIP Best PM",
            'Govt good', 'Govt bad',  'Govt net', 'Govt Question',
            "Pred Party Share: Con","Pred Party Share: Lab","Pred Party Share: LD","Pred Party Share: UKIP", "Pred Party Share: Question",

             ]
# #         # Con good	Con bad	Con net	Lab good	Lab bad	Lab net	LD good	LD bad	LD net->question!

# drop empty rows
df.drop(df.index[df.isnull().all(axis=1)],inplace=True)

df["Year"] = df["Year"].fillna( method ='ffill',axis=0 ).astype('int')
df["Month"] = df["Month"].fillna( method ='ffill',axis=0 )

df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Start_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 0) ).astype("float").astype("Int64") )
df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Finish_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 1) ).astype("float").astype("Int64") )          
df.drop("Fieldwork",axis=1,inplace=True)

df["Fieldwork_Start_Day"] = df["Unnamed: 3"].apply(lambda x: x.day)
df.drop("Unnamed: 3",axis=1,inplace=True)

# ## Manual fixes("")

# 2015GE = 7 May 2015
df.loc[df["Polling"]=="Result",["Year","Month","Fieldwork_Start_Day","Fieldwork_Finish_Day","Publication Day"]]=\
    [2015,"May",7,7,7]
# df["Publication Day"] = df["Publication Day"].apply(lambda x: x.split("/")[0] if pd.notnull(x) and not isinstance(x,int) else x)

df["Publication Day"] = df["Publication Day"].astype("float").astype("Int64")
df_list.append(df)
df        

10-15


,Year,Month,Fieldwork_Start_Day,Fieldwork_Finish_Day,Publication Day,Polling,Publisher,Con,Con Error,Lab,...,UKIP Best PM,Govt good,Govt bad,Govt net,Govt Question,Pred Party Share: Con,Pred Party Share: Lab,Pred Party Share: LD,Pred Party Share: UKIP,Pred Party Share: Question
0,2010,May,12.0,13,16,ComRes,IoS/SM,38.0,NaN,34.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010,May,12.0,13,16,ICM,STel,38.0,NaN,33.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2010,May,13.0,14,16,YouGov,ST,37.0,NaN,34.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2010,May,14.0,16,19,Angus RS,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2010,May,19.0,19,20,YouGov,Sun,NaN,NaN,NaN,...,NaN,43.0,23.0,20.0,Do you approve or disapprove of the Government...,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2090,2015,May,5.0,7,7,Populus,NaN,33.0,-1.0,33.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2091,2015,May,6.0,6,7,ComRes,DM/ITV,35.0,0.0,34.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2092,2015,May,6.0,6,NaN,Survation,NaN,37.0,NaN,31.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2094,2015,May,NaN,NaN,NaN,Exit poll,NaN,38.0,NaN,31.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [332]:
sheet_name = xl.sheet_names[20]
print(sheet_name)
df = xl.parse(sheet_name).replace(" ",np.nan)
import datetime
## Manual Fix

##### leadership ratings order isn't clear!
# assume same as others?
# Con good	Con bad	Con net	Lab good	Lab bad	Lab net	LD good	LD bad	LD net->question!


# Ashcroft - did the fieldwork really take over a month?
# df.loc[df["Fieldwork"]=='1-5 Sept',["Fieldwork"]]= '1-5'
# # df.loc[df["Fieldwork"]=="29-1 ?",["Fieldwork"]]= "29-1"
# df.loc[df["Published"]=="20?",["Published"]]= "20"

# df.loc[df["Published"]=="23/24",["Published"]]= "23"
# df.loc[df["Published"]==datetime.datetime(2011, 9, 25, 0, 0),["Published"]]= "25"
# df.loc[df["Published"]==datetime.datetime(2012, 11, 21, 0, 0),["Published"]]= "21"
# df.loc[df["Published"]==datetime.datetime(2014, 1, 4, 0, 0),["Published"]]= "4"
# df.loc[df["Published"]==datetime.datetime(2013, 12, 26, 0, 0),["Published"]]= "26"

# # df.loc[df["Fieldwork"]=="10/12",["Fieldwork"]]= "10"
# # df.loc[df["Date"]=='7?',["Date"]]= '7'
# # df.loc[df["Date"]=='19??',["Date"]]= '19'


# # df.loc[df["Fieldwork"]=="11-16 or 5-10",["Fieldwork"]]= "11-16"
# # df.loc[df["Date"]=="8-9",["Date"]]= "8"



# # # # df
df = df[['Year', 'Month', 'Fieldwork', 'Unnamed: 3', 'Published', 'Polling', 'Publisher',
       'Con', 'Unnamed: 9',
       'Lab', 'Unnamed: 11',
       'LD', 'Unnamed: 13',
       'UKIP', 'Unnamed: 15',
       'Green', 'Unnamed: 17',
       'Con lead', 'Unnamed: 19', 
       'Con good', 'Con bad', 'Con net', 'Lab good', 'Lab bad',
       'Lab net', 'LD good', 'LD bad', 'LD net', 'UKIP good', 'UKIP bad',
       'UKIP net', 'Grn good', 'Grn bad', 'Grn net', 'Question', 'Con Best PM',
       'Lab Best PM', 'LD Best PM', 'UKIP Best PM', 'Question.1', 'Govt good',
       'Govt bad', 'Govt net', 'Question.2', ]]

df.columns = ['Year', 'Month', 'Fieldwork', 'Unnamed: 3', 'Publication Day', 'Polling', 'Publisher',
              'Con', 'Con Error',
              'Lab', 'Lab Error',
              'LD', 'LD Error',
              'UKIP', 'UKIP Error',
              'Green', 'Green Error',
              'Con lead', 'Con lead Error',
            "Leader: Con good","Leader: Con bad","Leader: Con net",
            "Leader: Lab good","Leader: Lab bad","Leader: Lab net",
            "Leader: LD good", "Leader: LD bad", "Leader: LD net",
            "Leader: UKIP good","Leader: UKIP bad","Leader: UKIP net",
            "Leader: Grn good","Leader: Grn bad","Leader: Grn net",
            "Leader: Question",
            "Con Best PM","Lab Best PM","LD Best PM","UKIP Best PM","Best PM Question",
            'Govt good', 'Govt bad',  'Govt net', 'Govt Question',

             ]
# #         # Con good	Con bad	Con net	Lab good	Lab bad	Lab net	LD good	LD bad	LD net->question!

# drop empty rows
df.drop(df.index[df.isnull().all(axis=1)],inplace=True)

df["Year"] = df["Year"].fillna( method ='ffill',axis=0 ).astype('int')
df["Month"] = df["Month"].fillna( method ='ffill',axis=0 )

df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Start_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 0) ).astype("float").astype("Int64") )
df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Finish_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 1) ).astype("float").astype("Int64") )          
df.drop("Fieldwork",axis=1,inplace=True)

df["Fieldwork_Start_Day"] = df["Unnamed: 3"].apply(lambda x: x.day)
df.drop("Unnamed: 3",axis=1,inplace=True)

# ## Manual fixes("")

# 2017GE = 8 June 2017
df.loc[df["Polling"]=="Result",["Year","Month","Fieldwork_Start_Day","Fieldwork_Finish_Day","Publication Day"]]=\
    [2017,"Jun",8,8,8]
# df["Publication Day"] = df["Publication Day"].apply(lambda x: x.split("/")[0] if pd.notnull(x) and not isinstance(x,int) else x)

df["Publication Day"] = df["Publication Day"].astype("float").astype("Int64")
df_list.append(df)
df        

15-17


,Year,Month,Fieldwork_Start_Day,Fieldwork_Finish_Day,Publication Day,Polling,Publisher,Con,Con Error,Lab,...,Leader: Question,Con Best PM,Lab Best PM,LD Best PM,UKIP Best PM,Best PM Question,Govt good,Govt bad,Govt net,Govt Question
0,2015,May,8.0,9,10,Survation,MoS,40.0,NaN,31.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,May,14.0,15,NaN,YouGov,NaN,NaN,NaN,NaN,...,Do you think X is doing well or badly as…,NaN,NaN,NaN,NaN,NaN,39.0,46.0,-7.0,Do you approve or disapprove of the Government...
2,2015,May,21.0,22,NaN,YouGov,NaN,NaN,NaN,NaN,...,Do you think X is doing well or badly as…,NaN,NaN,NaN,NaN,NaN,42.0,45.0,-3.0,Do you approve or disapprove of the Government...
3,2015,May,25.0,26,1,YouGov,Sun,41.0,NaN,29.0,...,NaN,NaN,NaN,NaN,NaN,NaN,39.0,44.0,-5.0,Do you approve or disapprove of the Government...
4,2015,May,28.0,29,31,YouGov,ST,NaN,NaN,NaN,...,Do you think X is doing well or badly as…,NaN,NaN,NaN,NaN,NaN,40.0,44.0,-4.0,Do you approve or disapprove of the Government...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322,2017,Jun,6.0,7,8,BMG,The Herald,46.0,NaN,33.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
323,2017,Jun,6.0,7,7,Survation,NaN,41.0,-1.0,40.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
324,2017,Jun,6.0,7,8,Ipsos MORI,ES,44.0,-1.0,36.0,...,NaN,47.0,36.0,NaN,NaN,Who do you think would make the most capable P...,NaN,NaN,NaN,NaN
325,2017,Jun,7.0,8,NaN,Sky,NaN,NaN,NaN,NaN,...,Are you satisfied or disatisfied with the perf...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [333]:
sheet_name = xl.sheet_names[21]
print(sheet_name)
df = xl.parse(sheet_name).replace(" ",np.nan)
import datetime
## Manual Fix

##### leadership ratings order isn't clear!
# assume same as others?
# Con good	Con bad	Con net	Lab good	Lab bad	Lab net	LD good	LD bad	LD net->question!


# Ashcroft - did the fieldwork really take over a month?
df.loc[df["Year"]=='?',["Year"]]= np.nan
df.loc[df["Fieldwork"]=="29?",["Fieldwork"]]= "29"
df.loc[df["Fieldwork"]=="5?",["Fieldwork"]]= "5"

# df.loc[df["Published"]=="23/24",["Published"]]= "23"
df.loc[df["Published"]==datetime.datetime(2018, 12, 18, 0, 0),["Published"]]= "18"
# df.loc[df["Published"]==datetime.datetime(2012, 11, 21, 0, 0),["Published"]]= "21"
# df.loc[df["Published"]==datetime.datetime(2014, 1, 4, 0, 0),["Published"]]= "4"
# df.loc[df["Published"]==datetime.datetime(2013, 12, 26, 0, 0),["Published"]]= "26"

# # df.loc[df["Fieldwork"]=="10/12",["Fieldwork"]]= "10"
# # df.loc[df["Date"]=='7?',["Date"]]= '7'
# # df.loc[df["Date"]=='19??',["Date"]]= '19'


# # df.loc[df["Fieldwork"]=="11-16 or 5-10",["Fieldwork"]]= "11-16"
# # df.loc[df["Date"]=="8-9",["Date"]]= "8"



# # # # # df
df = df[['Year', 'Month', 'Fieldwork', 'Unnamed: 3', 'Published', 
       'Polling', 'Publisher',
       'Con', 'Unnamed: 9', 'Lab', 'Unnamed: 11', 'LD',
       'Unnamed: 13', 'UKIP', 'Unnamed: 15', 'Green', 'Unnamed: 17', 'TIG/CUK',
       'Unnamed: 19', 'BXP', 'Unnamed: 21', 'Con lead', 'Unnamed: 23',
       'Con good', 'Con bad', 'Con net',
       'Lab good', 'Lab bad', 'Lab net',
       'LD good', 'LD bad', 'LD net',
       'UKIP good', 'UKIP bad', 'UKIP net',
       'Grn good', 'Grn bad', 'Grn net',
       'CUK good', 'CUK bad', 'CUK net',
       'BXP good', 'BXP bad', 'BXP net', 'Question',
       'Con Best PM', 'Lab Best PM', 'LD Best PM', 'BXP Best PM', 'Question.1',
       'Govt good', 'Govt bad', 'Govt net', 'Question.2']]

df.columns = ['Year', 'Month', 'Fieldwork', 'Unnamed: 3', 'Publication Day',
              'Polling', 'Publisher',
              'Con', 'Con Error',
              'Lab', 'Lab Error',
              'LD', 'LD Error',
              'UKIP', 'UKIP Error',
              'Green', 'Green Error',
              'TIG/CUK', 'TIG/CUK Error',
              'BXP', 'BXP Error',
              'Con lead', 'Con lead Error',
            "Leader: Con good","Leader: Con bad","Leader: Con net",
            "Leader: Lab good","Leader: Lab bad","Leader: Lab net",
            "Leader: LD good", "Leader: LD bad", "Leader: LD net",
            "Leader: UKIP good","Leader: UKIP bad","Leader: UKIP net",
            "Leader: Grn good","Leader: Grn bad","Leader: Grn net",
            "Leader: CUK good","Leader: CUK bad","Leader: CUK net",
            "Leader: BXP good","Leader: BXP bad","Leader: BXP net",              
            "Leader: Question",
            "Con Best PM","Lab Best PM","LD Best PM","BXP Best PM","Best PM Question",
            'Govt good', 'Govt bad',  'Govt net', 'Govt Question',

             ]
# #         # Con good	Con bad	Con net	Lab good	Lab bad	Lab net	LD good	LD bad	LD net->question!

# drop empty rows
df.drop(df.index[df.isnull().all(axis=1)],inplace=True)

df["Year"] = df["Year"].fillna( method ='ffill',axis=0 ).astype('int')
df["Month"] = df["Month"].fillna( method ='ffill',axis=0 )

df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Start_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 0) ).astype("float").astype("Int64") )
df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Finish_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 1) ).astype("float").astype("Int64") )          
df.drop("Fieldwork",axis=1,inplace=True)

df["Fieldwork_Start_Day"] = df["Unnamed: 3"].apply(lambda x: x.day)
df.drop("Unnamed: 3",axis=1,inplace=True)

# ## Manual fixes("")

# 2019GE = 12 December 2019
df.loc[df["Polling"]=="Result",["Year","Month","Fieldwork_Start_Day","Fieldwork_Finish_Day","Publication Day"]]=\
    [2019,"Dec",12,12,12]
# df["Publication Day"] = df["Publication Day"].apply(lambda x: x.split("/")[0] if pd.notnull(x) and not isinstance(x,int) else x)

df["Publication Day"] = df["Publication Day"].astype("float").astype("Int64")
df_list.append(df)
df        

17-19


,Year,Month,Fieldwork_Start_Day,Fieldwork_Finish_Day,Publication Day,Polling,Publisher,Con,Con Error,Lab,...,Leader: Question,Con Best PM,Lab Best PM,LD Best PM,BXP Best PM,Best PM Question,Govt good,Govt bad,Govt net,Govt Question
0,2017,Jun,9.0,10,11,YouGov,ST,NaN,NaN,NaN,...,NaN,39.0,39.0,NaN,NaN,Which of the following do you think would make...,NaN,NaN,NaN,NaN
1,2017,Jun,10.0,10,11,Survation,MoS,39.0,NaN,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017,Jun,11.0,12,15,YouGov,NaN,NaN,NaN,NaN,...,Do you have a favourable or unfavourable opini...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017,Jun,15.0,16,NaN,YouGov,NaN,NaN,NaN,NaN,...,Do you have a favourable or unfavourable opini...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017,Jun,16.0,17,19,Survation,GMB,41.0,NaN,44.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537,2019,Dec,10.0,11,11,Opinium,NaN,45.0,-1.0,33.0,...,To what extent do you approve or disapprove of...,36.0,23.0,NaN,NaN,"Which, if any, of the following people do you ...",NaN,NaN,NaN,NaN
538,2019,Dec,10.0,11,11,Opinium,NaN,NaN,NaN,NaN,...,NaN,30.0,21.0,8.0,6.0,"Which, if any, of the following people do you ...",NaN,NaN,NaN,NaN
539,2019,Dec,10.0,11,11,Panelbase,NaN,43.0,0.0,34.0,...,"How good or bad a job do you think X, Y is doing?",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
540,2019,Dec,10.0,11,11,Survation,NaN,44.5,0.0,33.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [334]:
sheet_name = xl.sheet_names[22]
print(sheet_name)
df = xl.parse(sheet_name).replace(" ",np.nan)
import datetime
## Manual Fix

##### leadership ratings order isn't clear!
# assume same as others?
# Con good	Con bad	Con net	Lab good	Lab bad	Lab net	LD good	LD bad	LD net->question!


# Ashcroft - did the fieldwork really take over a month?
# df.loc[df["Year"]=='?',["Year"]]= np.nan
# df.loc[df["Fieldwork"]=="29?",["Fieldwork"]]= "29"
# df.loc[df["Fieldwork"]=="5?",["Fieldwork"]]= "5"

# # df.loc[df["Published"]=="23/24",["Published"]]= "23"
# df.loc[df["Published"]==datetime.datetime(2018, 12, 18, 0, 0),["Published"]]= "18"
# df.loc[df["Published"]==datetime.datetime(2012, 11, 21, 0, 0),["Published"]]= "21"
# df.loc[df["Published"]==datetime.datetime(2014, 1, 4, 0, 0),["Published"]]= "4"
# df.loc[df["Published"]==datetime.datetime(2013, 12, 26, 0, 0),["Published"]]= "26"

# # df.loc[df["Fieldwork"]=="10/12",["Fieldwork"]]= "10"
# # df.loc[df["Date"]=='7?',["Date"]]= '7'
# # df.loc[df["Date"]=='19??',["Date"]]= '19'


# # df.loc[df["Fieldwork"]=="11-16 or 5-10",["Fieldwork"]]= "11-16"
# # df.loc[df["Date"]=="8-9",["Date"]]= "8"



# # # # # # df
df = df[['Year', 'Month', 'Fieldwork', 'Unnamed: 3', 'Published', 
       'Polling', 'Publisher', 'Con', 'Unnamed: 9', 'Lab', 'Unnamed: 11', 'LD',
       'Unnamed: 13', 'Green', 'Unnamed: 15', 'BXP', 'Unnamed: 17', 'Con lead',
       'Unnamed: 19',
       'Con good', 'Con bad', 'Con net', 'Lab good', 'Lab bad', 'Lab net',
       'LD good', 'LD bad', 'LD net', 'BXP good', 'BXP bad', 'BXP net',
       'Grn good', 'Grn bad', 'Grn net', 'Question', 'Con Best PM',
       'Lab Best PM', 'LD Best PM', 'BXP Best PM', 'Question.1', 'Govt good',
       'Govt bad', 'Govt net', 'Question.2',]]

df.columns = ['Year', 'Month', 'Fieldwork', 'Unnamed: 3', 'Publication Day',
              'Polling', 'Publisher',
              'Con', 'Con Error',
              'Lab', 'Lab Error',
              'LD', 'LD Error',
              'Green', 'Green Error',
              'BXP', 'BXP Error',
              'Con lead', 'Con lead Error',
            "Leader: Con good","Leader: Con bad","Leader: Con net",
            "Leader: Lab good","Leader: Lab bad","Leader: Lab net",
            "Leader: LD good", "Leader: LD bad", "Leader: LD net",
            "Leader: BXP good","Leader: BXP bad","Leader: BXP net",                
            "Leader: Grn good","Leader: Grn bad","Leader: Grn net",
            "Leader: Question",
            "Con Best PM","Lab Best PM","LD Best PM","BXP Best PM","Best PM Question",
            'Govt good', 'Govt bad',  'Govt net', 'Govt Question',

             ]
# # #         # Con good	Con bad	Con net	Lab good	Lab bad	Lab net	LD good	LD bad	LD net->question!

# drop empty rows
df.drop(df.index[df.isnull().all(axis=1)],inplace=True)

df["Year"] = df["Year"].fillna( method ='ffill',axis=0 ).astype('int')
df["Month"] = df["Month"].fillna( method ='ffill',axis=0 )

df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Start_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 0) ).astype("float").astype("Int64") )
df.insert(list(df.columns).index("Fieldwork"), "Fieldwork_Finish_Day" ,df["Fieldwork"].apply(lambda x: get_fieldwork_days(x, 1) ).astype("float").astype("Int64") )          
df.drop("Fieldwork",axis=1,inplace=True)

df["Fieldwork_Start_Day"] = df["Unnamed: 3"].apply(lambda x: x.day)
df.drop("Unnamed: 3",axis=1,inplace=True)

# ## Manual fixes("")

# # 2019GE = 12 December 2019
# df.loc[df["Polling"]=="Result",["Year","Month","Fieldwork_Start_Day","Fieldwork_Finish_Day","Publication Day"]]=\
#     [2019,"Dec",12,12,12]
# # df["Publication Day"] = df["Publication Day"].apply(lambda x: x.split("/")[0] if pd.notnull(x) and not isinstance(x,int) else x)

df["Publication Day"] = df["Publication Day"].astype("float").astype("Int64")
df_list.append(df)
df        

19-


,Year,Month,Fieldwork_Start_Day,Fieldwork_Finish_Day,Publication Day,Polling,Publisher,Con,Con Error,Lab,...,Leader: Question,Con Best PM,Lab Best PM,LD Best PM,BXP Best PM,Best PM Question,Govt good,Govt bad,Govt net,Govt Question
0,2019,Dec,13,14,17,YouGov,NaN,NaN,NaN,NaN,...,Do you have a favourable or unfavourable opini...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [300]:
# df["Publication Day"][ df["Publication Day"].apply(lambda x: not isinstance(x,int)) ].dropna()
# df.loc[220 ,"Publication Day"]
# df["Fieldwork"][ df["Fieldwork"].apply(lambda x: True if isinstance(x,str) and "?" in x else False) ]
# df["Publication Day"][ df["Publication Day"].apply(lambda x: True if isinstance(x,str) and "/" in x else False) ]
# df["Year"][ df["Year"].apply(lambda x: True if isinstance(x,str) and "?" in x else False) ]

In [342]:
better_ordered_columns = \
['Year', 'Month', 'Fieldwork_Start_Day', 'Fieldwork_Finish_Day', 'Publication Day', 'Polling', 'Publisher',
    
'Con','Con Error', 'Con Female', 'Con Male', 'Con Male-Female',
'Lab','Lab Error', 'Lab Female', 'Lab Male', 'Lab Male-Female',
'LD', 'LD Error',  'LD Female',  'LD Male',  'LD Male-Female', 
'BNP',
'Green', 'Green Error', 
'UKIP', 'UKIP Error',
'BXP',  'BXP Error', 
'Referendum', 'Referendum Error',
'SDP','SDP Error',
'TIG/CUK', 'TIG/CUK Error',
 
'Con lead', 'Con lead Error',
 
'Leader: Question',
       'Leader: BXP bad', 'Leader: BXP good', 'Leader: BXP net',
       'Leader: CUK bad', 'Leader: CUK good', 'Leader: CUK net',
       'Leader: Con bad', 'Leader: Con good', 'Leader: Con net',
       'Leader: Grn bad', 'Leader: Grn good', 'Leader: Grn net',
       'Leader: LD bad', 'Leader: LD good', 'Leader: LD net',
       'Leader: Lab bad', 'Leader: Lab good', 'Leader: Lab net',
       'Leader: UKIP bad', 'Leader: UKIP good',
       'Leader: UKIP net',
 
'Govt Question', 'Govt bad', 'Govt good', 'Govt net',
 
'Best PM Question','Con Best PM','Lab Best PM','LD Best PM','BXP Best PM','UKIP Best PM',
 
'Pred Party Share: Question','Pred Party Share: Con','Pred Party Share: Lab','Pred Party Share: LD','Pred Party Share: UKIP',
 
 ]

In [353]:
df_final = pd.concat(df_list)[better_ordered_columns]
df_final.index = range(0,df_final.shape[0])
df_final.to_csv(Pollbase_folder+"MarkPackPollBase_flat_v1.csv")